# Creating experiment templates through ESCALATE REST


In [1]:
#factor this out when it's imported into the client

import requests

class ESCALATEClient():
    """ESCALATE API Client"""
    
    def __init__(self, base_url, username, password):
        self.base_url = base_url
        self.username = username
        self.password = password
        self._token = None
        self._is_logged_in = False
        self._login()
        
    def _login(self):
        r_login = requests.post(f'{self.base_url}/api/login', 
                                data={'username': self.username, 
                                      'password': self.password})
        self._token = r_login.json()['token']
        self._token_header = {'Authorization': f'Token {self._token}'}
        self._is_logged_in = True
        
    def get(self, 
            endpoint='', 
            data=None, 
            parse_json=True, 
            content_type='application/json'):
        """Make GET request with `data` to `endpoint` in ESCALATE API
    
        return: (dict|list|requests.Response), bool
        """
        data = {} if data is None else data
        r = requests.get(f'{self.base_url}/api/{endpoint}',
                         params=data, 
                         headers={**self._token_header, 
                                  'content-type': content_type})
        if r.ok: 
            print('GET: OK')
        else: 
            print('GET: FAILED')
        
        if r.ok and parse_json:
            resp_json = r.json()        
            # handle cases: no vs one vs many results
            count = resp_json.get('count')
            if count is None or count == 0:
                return r.json()
            elif count == 1: 
                print('Found one resource, returning dict')
                return resp_json['results'][0]
            elif count >= 1: 
                print(f"Found {resp_json['count']} resources, returning list of dicts)")
                return r.json()['results']
        print('Returning response object')   
        return r
        
    def post(self, 
             endpoint, 
             data, 
             content_type='application/json'):
        """POST `data` to `endpoint`in ESCALATE API using `content_type`
        return: (dict|requests.Response), bool
        """
        
        if not self._is_logged_in:
            raise ValueError("Not logged in: cannot post")
        
        r = requests.api.post(
            f'{self.base_url}/api/{endpoint}', 
            json=data, 
            headers={**self._token_header,
                     'content-type': content_type})
        print(r)
        if r.ok: 
            print('POST: OK, returning new resource dict')
            return r.json()
        print('POST: FAILED, returning response object')
        return r
    
    def put(self, data, url=None, endpoint=None, resource_id=None):
        """Update a resource
        Either provide a url or an endpoint and resource id
        """
        if not ((url is not None) or (endpoint is not None and resource_id is not None)): 
            raise ValueError("Must specify either url or endpoint and resource_id")
            
        if url is None: 
            url = f'{self.base_url}/api/{endpoint}/{resource_id}' 
        r = requests.api.put(url, json=data, headers=self._token_header)
        return r
    
    def patch(self, data, url=None, endpoint=None, resource_id=None):
        """Update parts of a resource
        Either provide a url or an endpoint and resource id
        """
        if not ((url is not None) or (endpoint is not None and resource_id is not None)): 
            raise ValueError("Must specify either url or endpoint and resource_id")
            
        if url is None: 
            url = f'{self.base_url}/api/{endpoint}/{resource_id}' 
        r = requests.api.patch(url, json=data, headers=self._token_header)
        return r
        
    def delete(self, url=None, endpoint=None, resource_id=None):
        """Delete a resource
         Either provide a url or an endpoint and resource id
        """
        if not ((url is not None) or (endpoint is not None and resource_id is not None)): 
            raise ValueError("Must specify either url or endpoint and resource_id")
        if url is None: 
            url = f'{self.base_url}/api/{endpoint}/{resource_id}' 
        r = requests.api.delete(url, headers=self._token_header)
        return r
    
    def list_endpoints(self):
        return self.get()

In [ ]:
def search(self, 
        endpoint='',
        related_ep=None, #for cross-searches
        search_field='',
        criteria= '',
        data=None, #must be a list
        exact=False,
        negate=False,
        parse_json=True, 
        content_type='application/json'):
    
    if negate==False:

        if exact==True:
            if data == None:
                '''Returns all fields for exact match'''
                if related_ep == None:
                
                    r = requests.get(f'{self.base_url}/api/{endpoint}/?{search_field}={criteria}', 
                                 headers={**self._token_header, 
                                          'content-type': content_type})
                else: #cross-search 
                    r = requests.get(f'{self.base_url}/api/{endpoint}/?{related_ep}__{search_field}={criteria}', 
                                 headers={**self._token_header, 
                                          'content-type': content_type})

            else:
                '''Returns requested field(s) for exact match'''
                i=0
                data_string=''
                while i<len(data)-1:
                    data_string+=data[i]+','
                    i+=1
                data_string+=data[i]
                
                if related_ep ==None:
                
                    r = requests.get(f'{self.base_url}/api/{endpoint}/?{search_field}={criteria}&fields={data_string}', 
                                     headers={**self._token_header, 
                                              'content-type': content_type}) 
                else: #cross-search
                    
                    r = requests.get(f'{self.base_url}/api/{endpoint}/?{related_ep}__{search_field}={criteria}&fields={data_string}', 
                                     headers={**self._token_header, 
                                              'content-type': content_type}) 

        else:
            
            
            if data == None:
                '''Containment test; returns all fields'''
                
                if related_ep == None:
                    r = requests.get(f'{self.base_url}/api/{endpoint}/?{search_field}__icontains={criteria}', 
                                     headers={**self._token_header, 
                                              'content-type': content_type}) 
                else: #cross-search
                    r = requests.get(f'{self.base_url}/api/{endpoint}/?{related_ep}__{search_field}__icontains={criteria}', 
                                     headers={**self._token_header, 
                                              'content-type': content_type})
                
            else:
                '''Containment test; returns requested field(s)'''
                i=0
                data_string=''
                while i<len(data)-1:
                    data_string+=data[i]+','
                    i+=1
                data_string+=data[i]
                
                if related_ep==None:
                    r = requests.get(f'{self.base_url}/api/{endpoint}/?{search_field}__icontains={criteria}&fields={data_string}', 
                                     headers={**self._token_header, 
                                              'content-type': content_type}) 
                else: #cross-search
                    r = requests.get(f'{self.base_url}/api/{endpoint}/?{related_ep}__{search_field}__icontains={criteria}&fields={data_string}', 
                                         headers={**self._token_header, 
                                                  'content-type': content_type})
    else:
    #negations

        if exact==True:
            if data == None:
                '''Returns all fields for exact match'''
                if related_ep == None:
                
                    r = requests.get(f'{self.base_url}/api/{endpoint}/?{search_field}!={criteria}', 
                                 headers={**self._token_header, 
                                          'content-type': content_type})
                else: #cross-search 
                    r = requests.get(f'{self.base_url}/api/{endpoint}/?{related_ep}__{search_field}!={criteria}', 
                                 headers={**self._token_header, 
                                          'content-type': content_type})

            else:
                '''Returns requested field(s) for exact match'''
                i=0
                data_string=''
                while i<len(data)-1:
                    data_string+=data[i]+','
                    i+=1
                data_string+=data[i]
                
                if related_ep ==None:
                
                    r = requests.get(f'{self.base_url}/api/{endpoint}/?{search_field}={criteria}&fields!={data_string}', 
                                     headers={**self._token_header, 
                                              'content-type': content_type}) 
                else: #cross-search
                    
                    r = requests.get(f'{self.base_url}/api/{endpoint}/?{related_ep}__{search_field}!={criteria}&fields={data_string}', 
                                     headers={**self._token_header, 
                                              'content-type': content_type}) 

        else:
            
            
            if data == None:
                '''Containment test; returns all fields'''
                
                if related_ep == None:
                    r = requests.get(f'{self.base_url}/api/{endpoint}/?{search_field}__icontains!={criteria}', 
                                     headers={**self._token_header, 
                                              'content-type': content_type}) 
                else: #cross-search
                    r = requests.get(f'{self.base_url}/api/{endpoint}/?{related_ep}__{search_field}__icontains!={criteria}', 
                                     headers={**self._token_header, 
                                              'content-type': content_type})
                
            else:
                '''Containment test; returns requested field(s)'''
                i=0
                data_string=''
                while i<len(data)-1:
                    data_string+=data[i]+','
                    i+=1
                data_string+=data[i]
                
                if related_ep==None:
                    r = requests.get(f'{self.base_url}/api/{endpoint}/?{search_field}__icontains!={criteria}&fields={data_string}', 
                                     headers={**self._token_header, 
                                              'content-type': content_type}) 
                else: #cross-search
                    r = requests.get(f'{self.base_url}/api/{endpoint}/?{related_ep}__{search_field}__icontains!={criteria}', 
                                         headers={**self._token_header, 
                                                  'content-type': content_type})                       

    if r.ok: 
        print('GET: OK')
    else: 
        print('GET: FAILED')

    if r.ok and parse_json:
        resp_json = r.json()        
        # handle cases: no vs one vs many results
        count = resp_json.get('count')
        if count is None or count == 0:
            return r.json()
        elif count == 1: 
            print('Found one resource, returning dict')
            return resp_json['results'][0]
        elif count >= 1: 
            print(f"Found {resp_json['count']} resources, returning list of dicts)")
            return r.json()['results']
    print('Returning response object')   
    return r

First import the REST API Client.

In [ ]:
import escalateclient
from escalateclient import ESCALATEClient

Enter your ESCALATE username and password, and the port number at which the server is running. Then run the following cell. This will create an instance of the API Client and set up a token to interact with the REST API.

In [2]:
username= 'nsmina'
password= 'password11'
port='8000'

escalate = ESCALATEClient(
    f'http://localhost:{port}',
    username,
    password
)

To see all the model endpoints (this is also a good way to verify that you are connected):

In [3]:
escalate.list_endpoints()

GET: OK


{'action': 'http://localhost:8000/api/action/',
 'actiondef': 'http://localhost:8000/api/action-def/',
 'actionunit': 'http://localhost:8000/api/action-unit/',
 'actor': 'http://localhost:8000/api/actor/',
 'basebommaterial': 'http://localhost:8000/api/base-bom-material/',
 'billofmaterials': 'http://localhost:8000/api/bill-of-materials/',
 'bomcompositematerial': 'http://localhost:8000/api/bom-composite-material/',
 'bommaterial': 'http://localhost:8000/api/bom-material/',
 'calculation': 'http://localhost:8000/api/calculation/',
 'calculationdef': 'http://localhost:8000/api/calculation-def/',
 'condition': 'http://localhost:8000/api/condition/',
 'conditiondef': 'http://localhost:8000/api/condition-def/',
 'experiment': 'http://localhost:8000/api/experiment/',
 'experimentinstance': 'http://localhost:8000/api/experiment-instance/',
 'experimenttemplate': 'http://localhost:8000/api/experiment-template/',
 'experimenttype': 'http://localhost:8000/api/experiment-type/',
 'experimentwork

## Create an Experiment

In [4]:
r = escalate.post('experiment/', {'description': 'Workflow 1 Demo', 'ref_uid': 'wf1_demo'})

exp = r['url']
exp

<Response [201]>
POST: OK, returning new resource dict


'http://localhost:8000/api/experiment/4e7ec69d-b124-45a0-b5ca-8682cad3bb27/'

* Create workflows to go with the experiment. (These correspond to parts of the procedure)

In [5]:
wfs = {name: escalate.post('workflow/', 
                          {'experiment': exp, 
                           'description': name
                          })['url']
      for name in ['Reagent Preparation', 'Synthesis']
      }

wfs

<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict


{'Reagent Preparation': 'http://localhost:8000/api/workflow/0841e1c1-5aa3-4962-ae27-2d8b94bab5f6/',
 'Synthesis': 'http://localhost:8000/api/workflow/279458f4-e4c8-42cc-a6f9-22b91964d48a/'}

## Gather Materials

* Search the database using `Get()` with filters to determine if the materials exist. If not, `Post()` them

We need lead (II) iodide (PbI2), ammonium iodide (NH4I), Gamma-butyrolactone (GBL), and formic acid (HCOOH)

In [6]:
mat_name_searches = ['Lead (II) Iodide', 'Ammonium Iodide', 'Gamma-Butyrolactone', 'Formic Acid']

for i in mat_name_searches:  
    r = escalate.get('material', data={'fields': ['description', 'url'],
                                      'description': i}) 
    print(r)

GET: OK
{'count': 0, 'next': None, 'previous': None, 'results': []}
GET: OK
{'count': 0, 'next': None, 'previous': None, 'results': []}
GET: OK
Found one resource, returning dict
{'url': 'http://localhost:8000/api/material/b9ad5eeb-9ab8-458f-b7cf-e9200b7964ad/', 'description': 'Gamma-Butyrolactone'}
GET: OK
Found one resource, returning dict
{'url': 'http://localhost:8000/api/material/758d9a15-69d6-4439-9d58-087a46d3d890/', 'description': 'Formic Acid'}


Searching for lead (II) iodide by exact name returned no results; let's expand our search criteria and do a cross-search by InChI key in the material identifer table to verify whether it's in the inventory.

In [7]:
r = escalate.get('material', data={'fields': ['description', 'url'],
                                   'identifier__description': 'RQQRAHKHDFPBMC-UHFFFAOYSA-L'}) 
r

GET: OK
Found one resource, returning dict


{'url': 'http://localhost:8000/api/material/56665bf9-566d-41dc-b8dd-6f74174e7435/',
 'description': 'Lead Diiodide'}

So it does exist, just not under the name we expected.

For ammonium iodide, which doesn't exist, we have to `Post()`. Note we could also associate identifiers like molecular formula and InChI with our new material, but for the scope of this tutorial it's enough to just post it with a description.

In [8]:
escalate.post(endpoint='material/', 
              data={'description': 'Ammonium Iodide', 
                    'material_class': 'model',
                    'consumable': True} 
                     )

<Response [201]>
POST: OK, returning new resource dict


{'url': 'http://localhost:8000/api/material/e5e68e17-9c15-4d36-8e54-e6f27efafe00/',
 'uuid': 'e5e68e17-9c15-4d36-8e54-e6f27efafe00',
 'edocs': [],
 'tags': [],
 'notes': [],
 'property': [],
 'add_date': '2021-09-07T17:37:30.589089',
 'mod_date': '2021-09-07T17:37:30.589110',
 'description': 'Ammonium Iodide',
 'consumable': True,
 'material_class': 'model',
 'internal_slug': 'ammonium-iodide',
 'status': None,
 'actor': None,
 'identifier': [],
 'material_type': []}

* Collect the materials within an inventory

In [9]:
#make life easier and just use the exact names that were returned by our searches
mat_names = ['Lead Diiodide', 'Ammonium Iodide', 'Gamma-Butyrolactone', 'Formic Acid']

materials = {name: escalate.get('material', {'description': name})['url']
            for name in mat_names
            }

materials

GET: OK
Found one resource, returning dict
GET: OK
Found one resource, returning dict
GET: OK
Found one resource, returning dict
GET: OK
Found one resource, returning dict


{'Lead Diiodide': 'http://localhost:8000/api/material/56665bf9-566d-41dc-b8dd-6f74174e7435/',
 'Ammonium Iodide': 'http://localhost:8000/api/material/e5e68e17-9c15-4d36-8e54-e6f27efafe00/',
 'Gamma-Butyrolactone': 'http://localhost:8000/api/material/b9ad5eeb-9ab8-458f-b7cf-e9200b7964ad/',
 'Formic Acid': 'http://localhost:8000/api/material/758d9a15-69d6-4439-9d58-087a46d3d890/'}

In [10]:
inventory = escalate.post('inventory/', data={'description': 'Workflow 1 Demo'})['url']

inventory

<Response [201]>
POST: OK, returning new resource dict


'http://localhost:8000/api/inventory/4a75a47b-dc21-47ff-90b0-63c99ffbd01e/'

In [11]:
inventory_materials = {description: escalate.post('inventory-material/', 
                                                  {'description': description, 
                                                   'material': url,
                                                   'inventory': inventory})['url']
                       for description, url in materials.items()}

inventory_materials

<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict


{'Lead Diiodide': 'http://localhost:8000/api/inventory-material/9e5dc29b-e3eb-4e92-a5b6-734913d6e612/',
 'Ammonium Iodide': 'http://localhost:8000/api/inventory-material/4f84deb8-123f-4fd9-85f3-2594112d9239/',
 'Gamma-Butyrolactone': 'http://localhost:8000/api/inventory-material/dc090731-4464-481a-a27c-53ea0880116f/',
 'Formic Acid': 'http://localhost:8000/api/inventory-material/3615c1aa-8f53-4661-9369-391b663d72d2/'}

* Create a bill of materials from the inventory, for the specific experiment

In [12]:
bom = escalate.post('bill-of-materials/', {'description': 'Workflow 1 Demo', 
                                          'experiment': exp})['url']

bom_materials = {name: escalate.post('bom-material/', 
                                     {'bom': bom, 'inventory_material': im}
                                    )['url']
                 for name, im in inventory_materials.items()}

bom_materials

<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict


{'Lead Diiodide': 'http://localhost:8000/api/base-bom-material/2ab45132-0cc3-4262-a884-3a2db82fd624/',
 'Ammonium Iodide': 'http://localhost:8000/api/base-bom-material/e7e0a9d4-9d5e-4798-a2e1-496a6aedb1d4/',
 'Gamma-Butyrolactone': 'http://localhost:8000/api/base-bom-material/046a9bae-d9a9-4302-a08e-f8cc2e55e0d3/',
 'Formic Acid': 'http://localhost:8000/api/base-bom-material/471d0e15-d4fe-4757-b886-151d44ba7bf9/'}

## Gather Containers

Currently containers are treated as materials. So we follow the same process of `Post()`ing first to the materials endpoint, then to the inventory, then to the BOM

We'll need four stock containers (let's call them tubes) and one 96-well plate. We can use one overall plate material and just create 96 wells in the BOM. For the tubes, we can create a generic stock tube. We can assume all four tubes for our experiment are identical, material-wise, and just create several instances with different names in the BOM.

In [ ]:
tube_material = escalate.post('material/',
             {'description': 'tube', 'material_class':'model'})

In [34]:
plate_material = escalate.post('material/',
             {'description': '96 well plate', 'material_class':'model'})

<Response [201]>
POST: OK, returning new resource dict


* Collect the containers within an inventory

In [35]:
cont_names= ['tube', '96 well plate']

for name in cont_names:

    cont_im = escalate.post('inventory-material/', 
                  {'description': name, 
                   'material':escalate.get('material', data={'description': name})['url'],
                   'inventory': inventory})['url']
    
    print(cont_im)

GET: OK
Found one resource, returning dict
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/inventory-material/a7978e88-daa5-4c95-8b63-ec4c4725b71b/
GET: OK
Found one resource, returning dict
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/inventory-material/39514fe7-fa9a-4237-9483-b17e7a3749ac/


* Create a bill of materials from the inventory, for the specific experiment

In [38]:
tube_names=['Stock Tube 1', 'Stock Tube 2', 'Stock Container 3', 'Stock Container 4']

tube_inventory={}
for name in tube_names:
    tube_inventory[name]= escalate.get('inventory-material', data={'description': 'tube'})['url']
    
tube_inventory

GET: OK
Found one resource, returning dict
GET: OK
Found one resource, returning dict
GET: OK
Found one resource, returning dict
GET: OK
Found one resource, returning dict


{'Stock Tube 1': 'http://localhost:8000/api/inventory-material/a7978e88-daa5-4c95-8b63-ec4c4725b71b/',
 'Stock Tube 2': 'http://localhost:8000/api/inventory-material/a7978e88-daa5-4c95-8b63-ec4c4725b71b/',
 'Stock Container 3': 'http://localhost:8000/api/inventory-material/a7978e88-daa5-4c95-8b63-ec4c4725b71b/',
 'Stock Container 4': 'http://localhost:8000/api/inventory-material/a7978e88-daa5-4c95-8b63-ec4c4725b71b/'}

**NEED TO ADD DESCRIPTIONS WHEN I POST THESE**

In [41]:
bom_tubes = {name: escalate.post('bom-material/', 
                                     {'bom': bom, 
                                      'inventory_material': url}
                                    )['url']
                 for name, url in tube_inventory.items()}

bom_tubes

<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict


{'Stock Tube 1': 'http://localhost:8000/api/base-bom-material/198b3a32-dd37-44a9-aafb-3411bc05afd6/',
 'Stock Tube 2': 'http://localhost:8000/api/base-bom-material/82aa8f24-bd54-450e-bcdb-ea66d42017fc/',
 'Stock Container 3': 'http://localhost:8000/api/base-bom-material/3be9b368-f21e-49ff-b6a5-2ccada38a1ea/',
 'Stock Container 4': 'http://localhost:8000/api/base-bom-material/d5ac3fe5-356c-4411-a900-c63669bf23a4/'}

In [50]:
idx=[]
for i in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']:
    for j in range(1,13):
        idx.append(i + str(j))

plate_url=escalate.get('inventory-material', {'description': '96 well plate'})['url']

well_inventory={}
for well in idx:
    well_inventory[well]=plate_url

bom_wellss = {name: escalate.post('bom-material/', 
                                     {'bom': bom, 
                                      'inventory_material': url}
                                    )['url']
                 for name, url in well_inventory.items()}
   

GET: OK
Found one resource, returning dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST:

NameError: name 'bom_wells' is not defined

In [51]:
bom_wellss

{'A1': 'http://localhost:8000/api/base-bom-material/3429cdfb-a140-4a60-8825-f40142b1328f/',
 'A2': 'http://localhost:8000/api/base-bom-material/d22f6fef-8388-430a-bcff-fcf58a34cdff/',
 'A3': 'http://localhost:8000/api/base-bom-material/c0a710e3-4c09-4bde-9fa4-eef8f4505079/',
 'A4': 'http://localhost:8000/api/base-bom-material/599d7d7a-5942-4e5c-bcc5-94022e07e607/',
 'A5': 'http://localhost:8000/api/base-bom-material/f7bd3a08-4f85-4e60-b520-26fd43231518/',
 'A6': 'http://localhost:8000/api/base-bom-material/5a8589ad-5d4f-4236-a3bd-abeba58c23bd/',
 'A7': 'http://localhost:8000/api/base-bom-material/2317cebe-595b-464c-b3a0-4e1504603c48/',
 'A8': 'http://localhost:8000/api/base-bom-material/fe3e35e3-3a5f-4bd7-ae7f-f332179f94d3/',
 'A9': 'http://localhost:8000/api/base-bom-material/a509b29c-3d43-45ec-90b1-8d629700b630/',
 'A10': 'http://localhost:8000/api/base-bom-material/1dacbb4b-9843-4241-a782-2095738a83ad/',
 'A11': 'http://localhost:8000/api/base-bom-material/547ac41d-e574-449f-b106-0e

## Set Up Actions

* Determine all actions needed for the experiment procedure. Use `Get()` to search the database for definitions of these actions (action-def). `Post()` new ones as needed

In [52]:
action_def_names = ['dispense_liquid', 'dispense_solid', 'heat', 'cool', 'transfer', 'stir']

for i in action_def_names:  
    r = escalate.get('action-def', data={'fields': ['description', 'url'],
                                      'description': i}) 
    print(r)

GET: OK
{'count': 0, 'next': None, 'previous': None, 'results': []}
GET: OK
Found one resource, returning dict
{'url': 'http://localhost:8000/api/action-def/0ccac627-5e68-4010-84b4-d1db19adabf9/', 'description': 'dispense_solid'}
GET: OK
Found one resource, returning dict
{'url': 'http://localhost:8000/api/action-def/030d508f-49a4-4eab-a83b-fb1635f3e793/', 'description': 'heat'}
GET: OK
{'count': 0, 'next': None, 'previous': None, 'results': []}
GET: OK
{'count': 0, 'next': None, 'previous': None, 'results': []}
GET: OK
{'count': 0, 'next': None, 'previous': None, 'results': []}


Looks like we have definitions for heating and for dispensing solid ... there's heat-stir but not just stir, and the other ones are completely missing. So we `Post()`.

In [53]:
missing = ['dispense_liquid', 'cool', 'transfer', 'stir']

for i in missing:
    r = escalate.post('action-def/', {'description': i, 
                                          'status': escalate.get('status', {'description':'test'})['url']}
                         )

GET: OK
Found one resource, returning dict
<Response [201]>
POST: OK, returning new resource dict
GET: OK
Found one resource, returning dict
<Response [201]>
POST: OK, returning new resource dict
GET: OK
Found one resource, returning dict
<Response [201]>
POST: OK, returning new resource dict
GET: OK
Found one resource, returning dict
<Response [201]>
POST: OK, returning new resource dict


In [54]:
actions = {
    name: escalate.get('action-def', data={'description': name})['url'] 
    for name in action_def_names}

actions

GET: OK
Found one resource, returning dict
GET: OK
Found one resource, returning dict
GET: OK
Found one resource, returning dict
GET: OK
Found one resource, returning dict
GET: OK
Found one resource, returning dict
GET: OK
Found one resource, returning dict


{'dispense_liquid': 'http://localhost:8000/api/action-def/60677fdb-6cd2-4239-9915-400e35ff184f/',
 'dispense_solid': 'http://localhost:8000/api/action-def/0ccac627-5e68-4010-84b4-d1db19adabf9/',
 'heat': 'http://localhost:8000/api/action-def/030d508f-49a4-4eab-a83b-fb1635f3e793/',
 'cool': 'http://localhost:8000/api/action-def/7abb21cd-1fff-495c-ace9-e5d81ecf99af/',
 'transfer': 'http://localhost:8000/api/action-def/e270c81e-b94d-49ae-b68f-2e8a86c31ee8/',
 'stir': 'http://localhost:8000/api/action-def/bf3bcb09-0a6e-4006-8688-73586036ff85/'}

* Determine the parameters (e.g. temperature, stir speed) that you'll need to specify for each action. Search if these exist, and post new ones as needed

We want temperature for heating/cooling, speed/duration for stirring, and mass and volume for the dispense and transfer actions. Let's use the case-insensitive containtment filter `__icontains` for our search because we don't know exactly what the parameter definitions are called.

In [55]:
param_names = ['temperature', 'speed', 'duration', 'mass', 'volume']

for i in param_names:  
    r = escalate.get('parameter-def', data={'fields': ['description', 'url'],
                                      'description__icontains': i}) 
    print(r)

GET: OK
Found one resource, returning dict
{'url': 'http://localhost:8000/api/parameter-def/e4db0901-aabd-4abe-87a3-64a38d9a8217/', 'description': 'temperature'}
GET: OK
Found one resource, returning dict
{'url': 'http://localhost:8000/api/parameter-def/1e51d3cd-7cf6-4765-bfc1-e7d54540696f/', 'description': 'speed'}
GET: OK
Found one resource, returning dict
{'url': 'http://localhost:8000/api/parameter-def/2cc0ed35-abaf-4ea5-b4c2-c848b0359129/', 'description': 'duration'}
GET: OK
Found one resource, returning dict
{'url': 'http://localhost:8000/api/parameter-def/2d9eb6b7-9e17-4e5f-855f-656687c80327/', 'description': 'mass'}
GET: OK
Found 6 resources, returning list of dicts)
[{'url': 'http://localhost:8000/api/parameter-def/79df4ee7-776f-4544-9636-6c37df64cc96/', 'description': 'volume'}, {'url': 'http://localhost:8000/api/parameter-def/73dfd457-1b05-490c-9d6f-3d6cbcce3ad3/', 'description': 'Sample Solvent Volume'}, {'url': 'http://localhost:8000/api/parameter-def/b701f73d-74b3-48ad-be

Looks like there are several volume-related parameters; the one we want is just "volume".

* Associate the action definitions and parameter definitions using `Patch()`

In [56]:
escalate.patch(url=actions['heat'], 
             data={'parameter_def': [escalate.get('parameter-def', 
                                                  {'description': 'temperature'})['url'], 
                                    ]
                  })

escalate.patch(url=actions['cool'], 
             data={'parameter_def': [escalate.get('parameter-def', 
                                                  {'description': 'temperature'})['url'],
                                    ]
                  })

escalate.patch(url=actions['stir'], 
             data={'parameter_def': [escalate.get('parameter-def', 
                                                  {'description': 'speed'})['url'],
                                     escalate.get('parameter-def', 
                                                  {'description': 'duration'})['url'] 
                                    ]
                  })

escalate.patch(url=actions['dispense_solid'], 
             data={'parameter_def': [escalate.get('parameter-def', 
                                                  {'description': 'mass'})['url'],
                                    ]
                  })

escalate.patch(url=actions['dispense_liquid'], 
             data={'parameter_def': [escalate.get('parameter-def', 
                                                  {'description': 'volume'})['url'],
                                    ]
                  })

escalate.patch(url=actions['transfer'], 
             data={'parameter_def': [escalate.get('parameter-def', 
                                                  {'description': 'mass'})['url'],
                                     escalate.get('parameter-def', 
                                                  {'description': 'volume'})['url'] 
                                    ]
                  })

GET: OK
Found one resource, returning dict
GET: OK
Found one resource, returning dict
GET: OK
Found one resource, returning dict
GET: OK
Found one resource, returning dict
GET: OK
Found one resource, returning dict
GET: OK
Found one resource, returning dict
GET: OK
Found one resource, returning dict
GET: OK
Found one resource, returning dict


<Response [200]>

**For transfer we can just have a volume param!**

* Create action instances for each action definition

(The instance is linked to an action definition (e.g. dispense solid) and to the desired workflow)

In [57]:
prep_actions = ['dispense_solid', 'dispense_liquid', 'cool', 'stir']

action_inst1={}

for action in prep_actions:
    r = escalate.post('action/', 
              {'workflow': wfs['Reagent Preparation'], 
               'action_def': actions[action],
              }
    )['url']
    
    action_inst1[action]=r
    
    print(r)

<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/action/cb509d82-18e9-453a-ac75-9951fd6f0225/
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/action/d68f6c57-61dd-4ea1-8075-a068cc347883/
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/action/27280834-dc27-4e8d-9047-d4635ed4cd26/
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/action/8232790d-3791-4a11-952f-6b2bdd5f230b/


In [58]:
synthesis_actions= ['heat', 'transfer', 'stir']

action_inst2={}

for action in synthesis_actions:
    r = escalate.post('action/', 
              {'workflow': wfs['Synthesis'], 
               'action_def': actions[action],
              }
    )['url']
    
    action_inst2[action]=r
        
    print(r)

<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/action/634146e1-9072-4749-b8d3-d95491a0b9d5/
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/action/9284c4fb-3e4d-4d94-9a77-ed04dc9b3ad5/
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/action/e5393a85-9b90-4eb7-8fce-2a73783fbc79/


## Implement steps of the procedure

* Generate action units for each step 

In [61]:
#reagent prep steps of workflow
#step 1: add 25 mL GBL to stock tube 1

r= escalate.post('action-unit/', 
              {'action': action_inst1['dispense_liquid'],
               'source_material': bom_materials['Gamma-Butyrolactone'], #material, from BOM
               'destination_material': bom_tubes['Stock Tube 1'] #container, from BOM
              })

<Response [201]>
POST: OK, returning new resource dict


For this action, we are dispensing liquid, so we want to know how much. That means we need to specify the (actual) value of the volume parameter. To do so, we use `Patch()`. Once this is done, if we navigate to the url of the specific action unit instance AND to the url of the parameter instance, we should see the (actual) parameter value for volume listed as 25 mL.

In [63]:
param_url= r['parameter'][0]['url']

param_url

patch_data = {'parameter_val_actual': {'value': 25, 'unit': 'mL', 'type': 'num'}}

r = escalate.patch(data=patch_data, url=param_url)

In [67]:
bom_tubes

{'Stock Tube 1': 'http://localhost:8000/api/base-bom-material/198b3a32-dd37-44a9-aafb-3411bc05afd6/',
 'Stock Tube 2': 'http://localhost:8000/api/base-bom-material/82aa8f24-bd54-450e-bcdb-ea66d42017fc/',
 'Stock Container 3': 'http://localhost:8000/api/base-bom-material/3be9b368-f21e-49ff-b6a5-2ccada38a1ea/',
 'Stock Container 4': 'http://localhost:8000/api/base-bom-material/d5ac3fe5-356c-4411-a900-c63669bf23a4/'}

In [68]:
#step 2: add 10 mL formic acid to Stock Container 4

r= escalate.post('action-unit/', 
              {'action': action_inst1['dispense_liquid'],
               'source_material': bom_materials['Formic Acid'], #material, from BOM
               'destination_material': bom_tubes['Stock Container 4'] #container, from BOM
              })

param_url= r['parameter'][0]['url']

patch_data = {'parameter_val_actual': {'value': 10, 'unit': 'mL', 'type': 'num'}}

r = escalate.patch(data=patch_data, url=param_url)

r

<Response [201]>
POST: OK, returning new resource dict


<Response [200]>

When we have a mixture of different materials in a container we will need several dispensing steps, one for each material. We can handle all the solids in a loop, since the action unit is of the same type.

In [72]:
#step 3 = add 0.5 g PbI2, 0.5 g NH4I, 10 ml GBL to Stock Tube 2

solids={'Lead Diiodide': 0.5, 'Ammonium Iodide': 0.5}
for key, val in solids.items():
    r= escalate.post('action-unit/', 
                  {'action': action_inst1['dispense_solid'],
                   'source_material': bom_materials[key], #material, from BOM
                   'destination_material': bom_tubes['Stock Tube 2'] #container, from BOM
                  })
    
    param_url= r['parameter'][0]['url']
    
    print(param_url)

    patch_data = {'parameter_val_actual': {'value': val, 'unit': 'g', 'type': 'num'}}
    
    r = escalate.patch(data=patch_data, url=param_url)

<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/parameter/c309b221-c8e5-49ac-a09e-4fe02c8a4312/
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/parameter/666fe60a-e62f-44c3-aa57-ca104a3ebc09/


In [73]:
r= escalate.post('action-unit/', 
          {'action': action_inst1['dispense_liquid'],
           'source_material': bom_materials['Gamma-Butyrolactone'], #material, from BOM
           'destination_material': bom_tubes['Stock Tube 2'] #container, from BOM
          })

param_url= r['parameter'][0]['url']

patch_data = {'parameter_val_actual': {'value': 10, 'unit': 'mL', 'type': 'num'}}

r = escalate.patch(data=patch_data, url=param_url)

r

<Response [201]>
POST: OK, returning new resource dict


<Response [200]>

In [77]:
#step 4 = add 1 g NH4I, 10 mL GBL to Stock Tube 3

r= escalate.post('action-unit/', 
          {'action': action_inst1['dispense_solid'],
           'source_material': bom_materials['Ammonium Iodide'], #material, from BOM
           'destination_material': bom_tubes['Stock Container 3'] #container, from BOM
          })

param_url= r['parameter'][0]['url']

patch_data = {'parameter_val_actual': {'value': 1, 'unit': 'g', 'type': 'num'}}

r = escalate.patch(data=patch_data, url=param_url)

r

<Response [201]>
POST: OK, returning new resource dict


<Response [200]>

In [76]:
r= escalate.post('action-unit/', 
          {'action': action_inst1['dispense_liquid'],
           'source_material': bom_materials['Gamma-Butyrolactone'], #material, from BOM
           'destination_material': bom_tubes['Stock Container 3'] #container, from BOM
          })

param_url= r['parameter'][0]['url']

patch_data = {'parameter_val_actual': {'value': 10, 'unit': 'mL', 'type': 'num'}}

r = escalate.patch(data=patch_data, url=param_url)

r

<Response [201]>
POST: OK, returning new resource dict


<Response [200]>

In [87]:
bom_tubes

{'Stock Tube 1': 'http://localhost:8000/api/base-bom-material/198b3a32-dd37-44a9-aafb-3411bc05afd6/',
 'Stock Tube 2': 'http://localhost:8000/api/base-bom-material/82aa8f24-bd54-450e-bcdb-ea66d42017fc/',
 'Stock Container 3': 'http://localhost:8000/api/base-bom-material/3be9b368-f21e-49ff-b6a5-2ccada38a1ea/',
 'Stock Container 4': 'http://localhost:8000/api/base-bom-material/d5ac3fe5-356c-4411-a900-c63669bf23a4/'}

In [86]:
#stir reagents in containers 2 and 3 until contents dissolve
conts=['Stock Tube 2', 'Stock Container 3']

for cont in conts:
    r= escalate.post('action-unit/', 
              {'action': action_inst1['stir'],
               'source_material': bom_tubes[cont], 
               'destination_material': None 
              })
    print(r)

<Response [201]>
POST: OK, returning new resource dict
{'url': 'http://localhost:8000/api/action-unit/766a6d8f-26ce-4d6f-9aea-b238cc078519/', 'uuid': '766a6d8f-26ce-4d6f-9aea-b238cc078519', 'edocs': [], 'tags': [], 'notes': [], 'parameter': [{'url': 'http://localhost:8000/api/parameter/cad773b9-4b6c-4acf-a2d0-61c9b6d39cb3/', 'uuid': 'cad773b9-4b6c-4acf-a2d0-61c9b6d39cb3', 'add_date': '2021-09-07T18:52:41.933830', 'mod_date': '2021-09-07T18:52:41.933849', 'parameter_val_nominal': {'value': 0.0, 'unit': 'mins', 'type': 'num'}, 'parameter_val_actual': {'value': 0.0, 'unit': 'mins', 'type': 'num'}, 'status': 'http://localhost:8000/api/status/3e3396c5-5bfe-4ecd-a72a-5e65f90dd04b/', 'actor': None, 'parameter_def': 'http://localhost:8000/api/parameter-def/2cc0ed35-abaf-4ea5-b4c2-c848b0359129/', 'action_unit': 'http://localhost:8000/api/action-unit/766a6d8f-26ce-4d6f-9aea-b238cc078519/'}, {'url': 'http://localhost:8000/api/parameter/a9cdbdda-5a5e-4360-b96f-6128272cb6eb/', 'uuid': 'a9cdbdda-5a5

In [90]:
#cool containers 2 and 3 to room temp

for cont in conts:
    r= escalate.post('action-unit/', 
              {'action': action_inst1['cool'],
               'source_material': bom_tubes[cont], 
               'destination_material': None 
              })
    
    param_url= r['parameter'][0]['url']

    patch_data = {'parameter_val_actual': {'value': 25, 'unit': 'deqC', 'type': 'num'}}

    r = escalate.patch(data=patch_data, url=param_url)

<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict


In [95]:
##synthesis portion of workflow
#heat well plate to 105 (actual temp is 95... how to distinguish this?)

for well in bom_wellss.keys():
    r= escalate.post('action-unit/', 
                  {'action': action_inst2['heat'],
                   'source_material': bom_wellss[well], 
                   'destination_material': None 
                  })
    
    param_url= r['parameter'][0]['url']
    
    #print(param_url)

    patch_data = {'parameter_val_nominal': {'value': 105, 'unit': 'deqC', 'type': 'num'}}

    r = escalate.patch(data=patch_data, url=param_url)
    
    patch_data = {'parameter_val_actual': {'value': 95, 'unit': 'deqC', 'type': 'num'}}
    
    r = escalate.patch(data=patch_data, url=param_url)

<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/parameter/5bcf6bd9-d53e-44da-b890-5f67636526ae/
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/parameter/0c86fd3c-15df-44d1-b38c-57cef8622513/
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/parameter/7483c06d-94d0-4a86-b157-ee94b97e8d7f/
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/parameter/fae8ba52-7ddb-416d-a71f-9acc60dc9705/
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/parameter/5fb42a6b-7ef6-4d36-885d-2890485d6bac/
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/parameter/55bec7ba-c22d-4538-897a-73f9520c8fb7/
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/parameter/f94373d2-5b60-43c0-9b17-2b9c4237a5b5/
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/parameter/29ec14

In [96]:
#add 1 microliter of reagent 1 to each of the plates 

for well in bom_wellss.keys():
    r= escalate.post('action-unit/', 
                  {'action': action_inst2['transfer'],
                   'source_material': bom_tubes['Stock Tube 1'], 
                   'destination_material': bom_wellss[well] 
                  })
    
    param_url= r['parameter'][0]['url']
    
    print(param_url)

    patch_data = {'parameter_val_actual': {'value': .001, 'unit': 'mL', 'type': 'num'}}
    
    r = escalate.patch(data=patch_data, url=param_url)

<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/parameter/b079abb9-2241-4c94-b479-798d7ec5b19e/
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/parameter/7672708b-cdfb-4097-8df5-ecf3ca688b5f/
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/parameter/b630a785-7964-4403-949f-91315327288f/
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/parameter/e3cb9367-0760-4d40-ac0f-512334375818/
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/parameter/4d1a57ca-ff92-4a4e-a437-543801b1f9a5/
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/parameter/ed1818dd-f44c-43c4-a55a-39ea0081e6a7/
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/parameter/80c6ecdd-c9d7-4916-b674-e16e8587ee1e/
<Response [201]>
POST: OK, returning new resource dict
http://localhost:8000/api/parameter/d7a8e0

In [97]:
#add 1 microliter of reagent 2 to each of the plates 

for well in bom_wellss.keys():
    r= escalate.post('action-unit/', 
                  {'action': action_inst2['transfer'],
                   'source_material': bom_tubes['Stock Tube 2'], 
                   'destination_material': bom_wellss[well] 
                  })
    
    param_url= r['parameter'][0]['url']

    patch_data = {'parameter_val_actual': {'value': .001, 'unit': 'mL', 'type': 'num'}}
    
    r = escalate.patch(data=patch_data, url=param_url)

#add 1 microliter of reagent 3 to each of the plates

for well in bom_wellss.keys():
    r= escalate.post('action-unit/', 
                  {'action': action_inst2['transfer'],
                   'source_material': bom_tubes['Stock Container 3'], 
                   'destination_material': bom_wellss[well] 
                  })
    
    param_url= r['parameter'][0]['url']

    patch_data = {'parameter_val_actual': {'value': .001, 'unit': 'mL', 'type': 'num'}}
    
    r = escalate.patch(data=patch_data, url=param_url)

#add .6 microliter of reagent 4 to each of the plates

for well in bom_wellss.keys():
    r= escalate.post('action-unit/', 
                  {'action': action_inst2['transfer'],
                   'source_material': bom_tubes['Stock Container 4'], 
                   'destination_material': bom_wellss[well] 
                  })
    
    param_url= r['parameter'][0]['url']

    patch_data = {'parameter_val_actual': {'value': .0006, 'unit': 'mL', 'type': 'num'}}
    
    r = escalate.patch(data=patch_data, url=param_url)

<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response 

In [ ]:
#stir well-plate for 15 min at 750 rpm

**need to distinguish between time and speed params in the index loop**

In [99]:
#add 0.6 microliters of reagent 4 to each of the plates

for well in bom_wellss.keys():
    r= escalate.post('action-unit/', 
                  {'action': action_inst2['transfer'],
                   'source_material': bom_tubes['Stock Container 4'], 
                   'destination_material': bom_wellss[well] 
                  })
    
    param_url= r['parameter'][0]['url']

    patch_data = {'parameter_val_actual': {'value': .0006, 'unit': 'mL', 'type': 'num'}}
    
    r = escalate.patch(data=patch_data, url=param_url)

<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict
<Response 

In [ ]:
#stir well-plate for 20 min at 750 rpm

* Associate action units with actions and workflows 

**I need help here**

## Gather Vessels

[pending]

In [ ]:
type_well96 = ContainerType(name='96-well PCR plate', is_tube=False, well_count=96, well_depth_mm=None, well_volume_ul=Unit(1.0, 'milliliter'), well_coating=None, sterile=None, cover_types=None, seal_types=['ultra-clear', 'foil'], capabilities=['liquid_handle', 'sangerseq', 'spin', 'thermocycle', 'incubate', 'gel_separate', 'gel_purify', 'seal', 'dispense'], shortname='96-pcr', col_count=12, dead_volume_ul=Unit(3, 'microliter'), safe_min_volume_ul=Unit(5, 'microliter'))

wellplate96 = ContainerModel(container_type=type_well96, name='96-well plate')


type_beaker50 = ContainerType(name='50 mL beaker', is_tube=True, well_count=1, well_depth_mm=None, well_volume_ul=Unit(50, 'milliliter'), well_coating=None, sterile=None, cover_types=None, seal_types=['ultra-clear', 'foil'], capabilities=['liquid_handle', 'sangerseq', 'spin', 'thermocycle', 'incubate', 'gel_separate', 'gel_purify', 'seal', 'dispense'], shortname='beaker', col_count=0, dead_volume_ul=Unit(3, 'microliter'), safe_min_volume_ul=Unit(5, 'microliter'))

beaker = ContainerModel(container_type=type_beaker50, name='50-mL beaker')


type_tube = ContainerType(name='centrifuge tube', is_tube=True, well_count=1, well_depth_mm=None, well_volume_ul=Unit(15, 'milliliter'), well_coating=None, sterile=None, cover_types=None, seal_types=['ultra-clear', 'foil'], capabilities=['liquid_handle', 'sangerseq', 'spin', 'thermocycle', 'incubate', 'gel_separate', 'gel_purify', 'seal', 'dispense'], shortname='beaker', col_count=0, dead_volume_ul=Unit(3, 'microliter'), safe_min_volume_ul=Unit(5, 'microliter'))

tube = ContainerModel(container_type=type_tube, name='centrifuge tube')


In [ ]:
for container in [tube, beaker]:
    
    r = escalate.post('vessel/', {'description': container.name, 
                                  'plate_name': 'null', 'well_number': 'null'}['url']
                     )

In [ ]:
r = escalate.post('vessel/', {'description': 'beaker', 'plate_name': 'null', 'well_number': 'null'})

so we've created a beaker using prototype ContainerModel and using conventional POST method to REST API

In [ ]:
for well in wellplate96.wells:
    
    r = escalate.post('vessel/', {'description': 'null', 
                                  'plate_name': '96wellplate', 'well_number': well.index}
                     )

In [ ]:
r = escalate.get('vessel',
            data={'fields': ['url', 'plate_name', 'well_number'],
                  'plate_name__icontains': '96wellplate'})
r

In [ ]:
vessels={}

r = escalate.get('vessel',
            data={'fields': ['url', 'plate_name', 'well_number'],
                  'plate_name__icontains': '96wellplate'})
for i in r:
    vessels[i['well_number']]=i['url']
    

r = escalate.get('vessel',
            data={'fields': ['url', 'description'],
                  'description__icontains': '50-mL beaker'})

vessels[r['description']]=r['url']


r = escalate.get('vessel',
            data={'fields': ['url', 'description'],
                  'description__icontains': 'tube'})

vessels[r['description']]=r['url']

vessels

we need  a second beaker and a second tube

In [ ]:
for container in [tube, beaker]:
    
    r = escalate.post('vessel/', {'description': container.name+'2', 
                                  'plate_name': 'null', 'well_number': 'null'}
                     )

In [ ]:
r = escalate.get('vessel',
            data={'fields': ['url', 'description'],
                  'description__icontains': '50-mL beaker'})

r

In [ ]:
vessels={}

r = escalate.get('vessel',
            data={'fields': ['url', 'plate_name', 'well_number'],
                  'plate_name__icontains': '96wellplate'})
for i in r:
    vessels[i['well_number']]=i['url']
    

r = escalate.get('vessel',
            data={'fields': ['url', 'description'],
                  'description__icontains': '50-mL beaker'})

for i in r:
    vessels[i['description']]=i['url']


r = escalate.get('vessel',
            data={'fields': ['url', 'description'],
                  'description__icontains': 'tube'})

for i in r:
    vessels[i['description']]=i['url']

vessels